In [ ]:
import pandas as pd
import os
import numpy as np

data_path = '../data'
question_info = pd.read_csv(os.path.join(data_path, 'question_info_0926.txt'), header=None, sep='\t')
answer_info = pd.read_csv(os.path.join(data_path, 'answer_info_0926.txt'), header=None, sep='\t')

answer_info.columns = ['回答id', '问题id', '用户id', '回答创建时间', '回答内容的单字编码序列', '回答内容的切词编码序列', '回答是否被标优', '回答是否被推荐', '回答是否被收入圆桌', '是否包含图片', '是否包含视频', '回答字数', '点赞数', '取赞数', '评论数', '收藏数', '感谢数', '举报数', '没有帮助数', '反对数']
question_info.columns = ['问题id','问题创建时间','问题标题单字编码','问题标题切词编码','问题描述单字编码','问题描述切词编码','问题绑定话题']

answer_info = pd.merge(answer_info, question_info, how='left', on='问题id')

drop_feat = ['回答内容的单字编码序列','回答内容的切词编码序列']
answer_info  = answer_info.drop(drop_feat, axis=1)

In [ ]:
print(answer_info.info())

In [ ]:
answer_info['回答创建时间-day'] = answer_info['回答创建时间'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)
answer_info['回答创建时间-hour'] = answer_info['回答创建时间'].apply(lambda x:x.split('-')[1].split('H')[1]).astype(int)

In [ ]:
# 减少内存占用
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
print(answer_info.info())

In [ ]:
answer_info['问题创建时间-day'] = answer_info['问题创建时间'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)
answer_info['问题创建时间-hour'] = answer_info['问题创建时间'].apply(lambda x:x.split('-')[1].split('H')[1]).astype(int)

In [ ]:
answer_info['用户回答与问题创建时间距离-day'] = answer_info['回答创建时间-day'] - answer_info['问题创建时间-day']
answer_info.loc[answer_info['用户回答与问题创建时间距离-day']==None, '用户回答与问题创建时间距离-day'] = 100
t3 = answer_info[answer_info['用户回答与问题创建时间距离-day']<100]
t3 = t3.groupby('用户id')['用户回答与问题创建时间距离-day'].agg(['mean', 'sum', 'max', 'min']).reset_index()  
t3.columns = ['用户id', 'u_qa_diffday_mean', 'u_qa_diffday_sum', 'u_qa_diffday_max', 'u_qa_diffday_min']
print(t3.info())

In [ ]:
def extract_user_qa_diff_day(answer_info):
    answer_info['用户回答与问题创建时间距离-day'] = answer_info['回答创建时间-day'] - answer_info['问题创建时间-day']
    answer_info.loc[answer_info['用户回答与问题创建时间距离-day']==None, '用户回答与问题创建时间距离-day'] = 100
    t3 = answer_info[answer_info['用户回答与问题创建时间距离-day']<100]
    t3 = t3.groupby('用户id')['用户回答与问题创建时间距离-day'].agg(['mean', 'sum', 'max', 'min']).reset_index()  
    t3.columns = ['用户id', 'u_qa_diffday_mean', 'u_qa_diffday_sum', 'u_qa_diffday_max', 'u_qa_diffday_min']
    #answer_info = pd.merge(answer_info, t3, on='用户id', how='left')
    
    #answer_info = reduce_mem_usage(answer_info)
    return t3

train_start = 3807
train_end = 3860
val_start = train_start + 7
val_end = train_end + 7

train_for_save1 = answer_info[answer_info['回答创建时间-day']>=train_start][answer_info['回答创建时间-day']<=train_end]
train_for_save1 = extract_user_qa_diff_day(train_for_save1)

val_for_save1 = answer_info[answer_info['回答创建时间-day']>=val_start][answer_info['回答创建时间-day']<=val_end]
val_for_save1 = extract_user_qa_diff_day(val_for_save1)

In [ ]:
print(train_for_save1)

In [ ]:
train_for_save1.to_hdf('u_qa_day_feats_train.h5', key='data')
val_for_save1.to_hdf('u_qa_day_feats_val.h5', key='data')

In [ ]:
train1 = pd.read_csv(os.path.join(data_path, 'invite_info_0926.txt'), header=None, sep='\t')
train1.columns = ['问题id', '用户id', '邀请创建时间','是否回答']
train1['邀请创建时间-day'] = train1['邀请创建时间'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)
train1['邀请创建时间-hour'] = train1['邀请创建时间'].apply(lambda x:x.split('-')[1].split('H')[1]).astype(int)

answer_info = pd.merge(answer_info, train1, how='left', on=['用户id','问题id'])

In [ ]:
print(answer_info.info())

In [ ]:
def extract_question_features(info,end):
    
    info['q_a_num'] = info['问题id'].map(info['回答id'].groupby(info['问题id']).nunique())
    info['q_a_num_last3'] = info['问题id'].map(info[info['邀请创建时间-day']>(end-3)]['回答id'].groupby(info['问题id']).nunique())
    info['q_a_num_last7'] = info['问题id'].map(info[info['邀请创建时间-day']>(end-7)]['回答id'].groupby(info['问题id']).nunique())
    info['q_a_num_last14'] = info['问题id'].map(info[info['邀请创建时间-day']>(end-14)]['回答id'].groupby(info['问题id']).nunique())
    
    info['问题最近7天回答比例'] = info['问题id'].map(info[info['邀请创建时间-day']>(end-7)]['回答id'].groupby(info['问题id']).nunique())/info['问题关联回答数']
    info['问题最近3天回答比例'] = info['问题id'].map(info[info['邀请创建时间-day']>(end-3)]['回答id'].groupby(info['问题id']).nunique())/info['问题关联回答数']
    info['问题最近14天回答比例'] = info['问题id'].map(info[info['邀请创建时间-day']>(end-14)]['回答id'].groupby(info['问题id']).nunique())/info['问题关联回答数']

    info['问题上次回答时间'] = info['问题id'].map(info['回答创建时间-day'].groupby(info['问题id']).max())
    
#     feat_cols = ['点赞数', '取赞数', '评论数', '收藏数', '感谢数', '举报数', '没有帮助数', '反对数','回答是否被标优', '回答是否被推荐', '回答是否被收入圆桌', '是否包含图片', '是否包含视频', '回答字数']
#     for feat in feat_cols:
#         info['问题总'+feat+'_last3week'] = info['用户id'].map(info[feat].groupby(info['问题id']).sum())
#         info['问题平均'+feat+'_last3week'] = info['用户id'].map(info[feat].groupby(info['问题id']).mean())
#         info['问题最大'+feat+'_last3week'] = info['用户id'].map(info[feat].groupby(info['问题id']).max())
#     #print(info[['问题id','']])
        
#     for feat in feat_cols:
#         info['问题总'+feat+'_last3day'] = info['用户id'].map(info[info['邀请创建时间-day']>end-3][feat].groupby(info['问题id']).sum())
#         info['问题平均'+feat+'_last3day'] = info['用户id'].map(info[info['邀请创建时间-day']>end-3][feat].groupby(info['问题id']).mean())
#         info['问题最大'+feat+'_last3day'] = info['用户id'].map(info[info['邀请创建时间-day']>end-3][feat].groupby(info['问题id']).max())
        
#     for feat in feat_cols:
#         info['问题总'+feat+'_last1week'] = info['用户id'].map(info[info['邀请创建时间-day']>end-7][feat].groupby(info['问题id']).sum())
#         info['问题平均'+feat+'_last1week'] = info['用户id'].map(info[info['邀请创建时间-day']>end-7][feat].groupby(info['问题id']).mean())
#         info['问题最大'+feat+'_last1week'] = info['用户id'].map(info[info['邀请创建时间-day']>end-7][feat].groupby(info['问题id']).max())
        
#     for feat in feat_cols:
#         info['问题总'+feat+'_last2week'] = info['用户id'].map(info[info['邀请创建时间-day']>end-14][feat].groupby(info['问题id']).sum())
#         info['问题平均'+feat+'_last2week'] = info['用户id'].map(info[info['邀请创建时间-day']>end-14][feat].groupby(info['问题id']).mean())
#         info['问题最大'+feat+'_last2week'] = info['用户id'].map(info[info['邀请创建时间-day']>end-14][feat].groupby(info['问题id']).max())
    
    drop_feat = ['用户回答与问题创建时间距离-day','回答id','用户id','邀请创建时间','邀请创建时间-day','邀请创建时间-hour','是否回答','回答创建时间','回答是否被标优','回答是否被推荐','回答是否被收入圆桌','点赞数','评论数','收藏数','感谢数','问题创建时间','问题标题单字编码',
            '问题标题切词编码','问题描述单字编码','问题描述切词编码','问题绑定话题','回答创建时间-day','回答创建时间-hour','取赞数','举报数', '没有帮助数', '反对数','回答是否被标优', '回答是否被推荐', '回答是否被收入圆桌', '是否包含图片', '是否包含视频', '回答字数']
    info = info.drop(drop_feat, axis=1)
    info.drop_duplicates(subset=None, keep='first', inplace=True)
    
    info = info.fillna(0)
    info = reduce_mem_usage(info)

    print(info.info())
    
    return info

In [ ]:
train_start = 3807
train_end = 3860
val_start = train_start + 7
val_end = train_end + 7

train_for_save = answer_info[answer_info['邀请创建时间-day']>=train_start][answer_info['邀请创建时间-day']<=train_end]
train_for_save = extract_question_features(train_for_save, train_end)

val_for_save = answer_info[answer_info['邀请创建时间-day']>=val_start][answer_info['邀请创建时间-day']<=val_end]
val_for_save = extract_question_features(val_for_save, val_end)

In [ ]:
print(train_for_save['问题id'].nunique())

In [ ]:
train_for_save.to_hdf('question_answer_train.h5', key='data')
val_for_save.to_hdf('question_answer_val.h5', key='data')

## ---分割线---

In [ ]:
import pandas as pd
import os
qinfo = pd.read_hdf('question_answer.h5', key='data')

In [ ]:
print(qinfo.info())

In [ ]:
print(info_for_save['问题id'].nunique())

In [ ]:
qinfo.drop_duplicates(inplace=True)

In [ ]:
print(qinfo.info())

In [ ]:
qinfo.to_hdf('question_answer.h5', key='data')